## Locais votação

Nesse notebook são extraídos os locais de votação de SP do TSE, tratados e geolocalizados com AzureMaps

In [1]:
import pandas as pd
import os
import zipfile
import requests
from io import BytesIO
from utils.save_csv import save_csv
from utils.geoloc import azure_maps_address_search
from config import data_folder

Extração

In [2]:
file_tse_locais = os.path.join(data_folder, 'locais_votacao.csv')

url_tse_locais = 'https://cdn.tse.jus.br/estatistica/sead/odsele/detalhe_votacao_secao/detalhe_votacao_secao_2024.zip'

In [3]:
def load_csv_votacao(url = url_tse_locais, file_name=file_tse_locais):

    print('Loading TSE voting locations data...')
    response = requests.get(url)

    with zipfile.ZipFile(BytesIO(response.content)) as z:

        csv_name = [f for f in z.namelist() if 'SP' in f and f.endswith('.csv')][0]

        with z.open(csv_name) as f:

            locais_votacao = pd.read_csv(f, 
                                         sep = ';', 
                                         encoding = 'latin1')

    save_csv(locais_votacao, file_name)

    return locais_votacao

In [4]:
if os.path.exists(file_tse_locais):

    locais_votacao = pd.read_csv(file_tse_locais,
                                 sep = ';', 
                                 encoding = 'utf-8')

else:
    print('Loading saved csv.')
    locais_votacao=load_csv_votacao()

Tratamento

In [5]:
locais_votacao_sp = locais_votacao[locais_votacao['NM_MUNICIPIO'] == 'SÃO PAULO']

In [6]:
locais_votacao_sp['NR_ZONA'].unique()

array([260, 246, 381, 389, 280, 346, 327, 374, 257, 403, 253, 392, 372,
       249, 376, 258, 353, 371, 421,   2, 348, 390, 349, 420, 326, 404,
       251,  20, 320, 413, 417, 254, 408, 328, 373, 325, 397, 405, 255,
       375, 418, 350, 256, 250, 347, 352,   6, 422,   4, 248,   3, 252,
         1, 351, 247,   5, 259])

In [7]:
locais_votacao_sp['NR_SECAO'].unique()

array([ 497,  499,  309, ..., 1049, 1057, 1034], shape=(1057,))

In [8]:
locais_votacao_sp['DS_LOCAL_VOTACAO_ENDERECO'].unique()

array(['RUA ALENCAR ARARIPE, 261', 'RUA CAMPANTE, 357',
       'RUA VERBO DIVINO, 993 A', ..., 'RUA RÔMULO NALDI, 117',
       'RUA ANTONIO MARIANI, 425', 'AV. NAÇÕES UNIDAS, 1525'],
      shape=(2033,), dtype=object)

Geolocalização

In [9]:
resp = [azure_maps_address_search(endereco) for endereco in locais_votacao_sp['DS_LOCAL_VOTACAO_ENDERECO'].unique()]

Searching Azure Maps:  https://atlas.microsoft.com/search/address/structured/json?streetName=RUA+ALENCAR+ARARIPE&streetNumber=261&municipality=Sao+Paulo&countrySecondarySubdivision=Sao+Paulo&countryCode=br&api-version=1.0&language=en-US&subscription-key=**TOKEN_STRING**
Searching Azure Maps:  https://atlas.microsoft.com/search/address/structured/json?streetName=RUA+CAMPANTE&streetNumber=357&municipality=Sao+Paulo&countrySecondarySubdivision=Sao+Paulo&countryCode=br&api-version=1.0&language=en-US&subscription-key=**TOKEN_STRING**
Searching Azure Maps:  https://atlas.microsoft.com/search/address/structured/json?streetName=RUA+VERBO+DIVINO&streetNumber=993&municipality=Sao+Paulo&countrySecondarySubdivision=Sao+Paulo&countryCode=br&api-version=1.0&language=en-US&subscription-key=**TOKEN_STRING**
Searching Azure Maps:  https://atlas.microsoft.com/search/address/structured/json?streetName=RUA+MARIA+ROSCHEL+SCHUNCK&streetNumber=690&municipality=Sao+Paulo&countrySecondarySubdivision=Sao+Paulo&

In [10]:
dados = []

for item in resp:
    original = item.get("summary", {}).get("query", None)
    results = item.get("results", [])

    if results:
        best = results[0]
        free = best.get("address", {}).get("freeformAddress")
        lat = best.get("position", {}).get("lat")
        lon = best.get("position", {}).get("lon")
        score = best.get("score")

    else:
        free = lat = lon = score = None

    dados.append({
        "end_original": original,
        "end_resultado": free,
        "lat": lat,
        "lon": lon,
        "score": score
    })

geo_locais = pd.DataFrame(dados)
geo_locais

,end_original,end_resultado,lat,lon,score
0,rua alencar araripe 261 sao paulo sao paulo,"Rua Alencar Araripe, 261, 04253-000, São Paulo",-23.609752,-46.597624,1.0
1,rua campante 357 sao paulo sao paulo,"Rua Campante, 357, 04224-010, São Paulo",-23.597247,-46.593080,1.0
2,rua verbo divino 993 sao paulo sao paulo,"Rua Verbo Divino, 993, 04719-905, São Paulo",-23.634526,-46.705804,1.0
3,rua maria roschel schunck 690 sao paulo sao paulo,"Rua Maria Roschel Schunck, 690, 04890-400, São...",-23.830547,-46.724916,1.0
4,rua guilherme correia de melo 408 sao paulo sa...,"Rua Guilherme Correia de Melo, 408, 05207-050,...",-23.399323,-46.751647,1.0
...,...,...,...,...,...
2028,rua mauro de araújo ribeiro 678 sao paulo sao ...,"Rua Mauro de Araújo Ribeiro, 678, 05182-000, S...",-23.449936,-46.744623,1.0
2029,rua barra do camarajibe 200 sao paulo sao paulo,"Rua Barra do Camarajibe, 200, 03383-120, São P...",-23.582316,-46.531668,1.0
2030,rua rômulo naldi 117 sao paulo sao paulo,"Rua Rômulo Naldi, 117, 02873-250, São Paulo",-23.455095,-46.683777,1.0
2031,rua antonio mariani 425 sao paulo sao paulo,"Rua Antônio Mariani, 425, 05530-000, São Paulo",-23.576427,-46.718505,1.0


In [12]:
geo_locais['score'].describe()

count    2033.000000
mean        0.987233
std         0.041599
min         0.425901
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: score, dtype: float64

In [21]:
geo_locais[geo_locais['score'] < 0.7]

,end_original,end_resultado,lat,lon,score
210,rua iguanara sao paulo sao paulo,São Paulo,-23.548666,-46.638252,0.505812
514,rua sapupira 1 005 sao paulo sao paulo,"Rua Sapupira, São Paulo",-23.503082,-46.469705,0.647760
558,rua dr adolpho de mello júnior 156 sao paulo s...,"Rua Doutor Alcides de Mello Ramalho, 156, 1208...",-23.028747,-45.538925,0.661517
840,rua gualdaquivir sao paulo sao paulo,São Paulo,-23.548666,-46.638252,0.505812
1288,av prof oscar campiglia sn em frente ao número...,"Rua Professor Oscar Campiglia, 272, 05762-230,...",-23.642862,-46.762167,0.425901
1333,rua boaventura rodrigues da silva 1 000 sao pa...,"Rua Boaventura Rodrigues da Silva, 03801-120, ...",-23.503174,-46.489723,0.697219


In [11]:
save_csv(geo_locais, 'locais_votacao_geoloc.csv')

Base salva em data\locais_votacao_geoloc.csv
